In [2]:
%load_ext autoreload
%autoreload 2

## Imports

In [1]:
import itertools
import warnings
from collections import Counter

import matplotlib as mpl
import numpy as np
import pandas as pd
import polars as pl
import seaborn as sns
from tqdm import tqdm

pd.options.display.max_columns = 100

mpl.rcParams["figure.max_open_warning"] = 0

In [2]:
assert pl.__version__ == "1.9.0"

# Read in data

## read in unfiltered data

The unfiltered data uses the fixed SCOP lookups provided by FoldSeek authors

In [3]:
pq = "s3://seanome-kmerseek/scope-benchmark/analysis-outputs/2024-10-09__hp_k20-60/00_cleaned_multisearch_results/scope40.multisearch.hp.k20.pq"
multisearch_unfiltered = pd.read_parquet(pq)

### Set SCOP lineage column names

In [17]:
lineage_cols = ["name", "family", "superfamily", "fold", "class"]
query_scop_cols = [f"query_{x}" for x in lineage_cols]
match_scop_cols = [f"match_{x}" for x in lineage_cols]

same_scop_cols = [f"same_{x}" for x in lineage_cols]

In [53]:
query_scop_cols_minus_name = [x for x in query_scop_cols if "name" not in x]
query_scop_cols_minus_name

['query_family', 'query_superfamily', 'query_fold', 'query_class']

In [55]:
lineage_cols_minus_name = [x for x in lineage_cols if "name" not in x]
lineage_cols_minus_name

['family', 'superfamily', 'fold', 'class']

### Make query metadata

In [18]:
query_metadata = pd.DataFrame(
    multisearch_unfiltered[query_scop_cols].values,
    index=multisearch_unfiltered["query_scop_id"].values,
    columns=query_scop_cols,
)
query_metadata = query_metadata.sort_index()
print(query_metadata.shape)
query_metadata = query_metadata.loc[~query_metadata.index.duplicated()]
print(query_metadata.shape)
query_metadata.head()

(3471675, 5)
(15177, 5)


,query_name,query_family,query_superfamily,query_fold,query_class
d12asa_,d12asa_ d.104.1.1 (A:) Asparagine synthetase {...,d.104.1.1,d.104.1,d.104,d
d16vpa_,d16vpa_ d.180.1.1 (A:) Conserved core of trans...,d.180.1.1,d.180.1,d.180,d
d1914a1,d1914a1 d.49.1.1 (A:4004-4081) Signal recognit...,d.49.1.1,d.49.1,d.49,d
d1914a2,d1914a2 d.49.1.1 (A:2001-2097) Signal recognit...,d.49.1.1,d.49.1,d.49,d
d1a04a1,d1a04a1 a.4.6.2 (A:150-216) Nitrate/nitrite re...,a.4.6.2,a.4.6,a.4,a


### Count number of groups per sample

In [45]:
def count_scop_lineage(df, col):
    return Counter(df[col])


n_groups_per_scop_lineage = {
    lineage: pd.Series(
        count_scop_lineage(query_metadata, f"query_{lineage}"), name=f"n_{lineage}"
    )
    for lineage in lineage_cols
    if lineage != "name"
}
n_groups_per_scop_lineage.keys()
# n_groups_per_scop_lineage

dict_keys(['family', 'superfamily', 'fold', 'class'])

In [46]:
n_groups_per_scop_lineage["class"]

d    3653
a    2644
c    4463
b    3059
f     332
g     722
e     304
Name: n_class, dtype: int64

In [47]:
query_metadata_with_n_groups = query_metadata.copy()

for lineage, series in n_groups_per_scop_lineage.items():
    # display(series)
    on = f"query_{lineage}"

    query_metadata_with_n_groups = query_metadata_with_n_groups.join(series, on=on)
query_metadata_with_n_groups.index.name = "query_scop_id"
query_metadata_with_n_groups.head()

,query_name,query_family,query_superfamily,query_fold,query_class,n_family,n_superfamily,n_fold,n_class
query_scop_id,,,,,,,,,
d12asa_,d12asa_ d.104.1.1 (A:) Asparagine synthetase {...,d.104.1.1,d.104.1,d.104,d,15,29,29,3653
d16vpa_,d16vpa_ d.180.1.1 (A:) Conserved core of trans...,d.180.1.1,d.180.1,d.180,d,1,1,1,3653
d1914a1,d1914a1 d.49.1.1 (A:4004-4081) Signal recognit...,d.49.1.1,d.49.1,d.49,d,2,3,3,3653
d1914a2,d1914a2 d.49.1.1 (A:2001-2097) Signal recognit...,d.49.1.1,d.49.1,d.49,d,2,3,3,3653
d1a04a1,d1a04a1 a.4.6.2 (A:150-216) Nitrate/nitrite re...,a.4.6.2,a.4.6,a.4,a,5,30,425,2644


In [48]:
query_metadata_with_n_groups.to_csv(
    "s3://seanome-kmerseek/scope-benchmark/reference_files/scop.e.2.08.query_metadata.csv"
)

### Add categories to make polars happy

In [54]:
for col in query_scop_cols_minus_name:
    categories = sorted(list(set(query_metadata_with_n_groups[col])))
    query_metadata_with_n_groups[col] = pd.Categorical(
        query_metadata_with_n_groups[col], ordered=True, categories=categories
    )
query_metadata_with_n_groups.dtypes

query_name             object
query_family         category
query_superfamily    category
query_fold           category
query_class          category
n_family                int64
n_superfamily           int64
n_fold                  int64
n_class                 int64
dtype: object

### Sort by query family, etc to make sure even physical ordering is lexicographical

In [78]:
query_metadata_with_n_groups = query_metadata_with_n_groups.sort_values(
    query_scop_cols_minus_name
)
query_metadata_with_n_groups

,query_name,query_family,query_superfamily,query_fold,query_class,n_family,n_superfamily,n_fold,n_class
query_scop_id,,,,,,,,,
d1x3ka_,d1x3ka_ a.1.1.0 (A:) automated matches {Tokuna...,a.1.1.0,a.1.1,a.1,a,22,57,62,2644
d1x46a_,d1x46a_ a.1.1.0 (A:) automated matches {Tokuna...,a.1.1.0,a.1.1,a.1,a,22,57,62,2644
d2bk9a_,d2bk9a_ a.1.1.0 (A:) automated matches {Fruit ...,a.1.1.0,a.1.1,a.1,a,22,57,62,2644
d2c0ka_,d2c0ka_ a.1.1.0 (A:) automated matches {Gaster...,a.1.1.0,a.1.1,a.1,a,22,57,62,2644
d2ig3a_,d2ig3a_ a.1.1.0 (A:) automated matches {Campyl...,a.1.1.0,a.1.1,a.1,a,22,57,62,2644
...,...,...,...,...,...,...,...,...,...
d6y3ba_,d6y3ba_ g.96.1.0 (A:) automated matches {Zika ...,g.96.1.0,g.96.1,g.96,g,1,2,2,722
d2foma1,d2foma1 g.96.1.1 (A:49-95) Flavivirus non-stru...,g.96.1.1,g.96.1,g.96,g,1,2,2,722
d3s2ra_,d3s2ra_ g.97.1.0 (A:) automated matches {Thale...,g.97.1.0,g.97.1,g.97,g,1,1,1,722


In [79]:
query_metadata_with_n_groups.to_parquet(
    "s3://seanome-kmerseek/scope-benchmark/reference_files/scop.e.2.08.query_metadata.pq"
)

### Read with polars to set the categorical ordering as lexical

In [80]:
query_metadata_pl = pl.read_parquet(
    "s3://seanome-kmerseek/scope-benchmark/reference_files/scop.e.2.08.query_metadata.pq"
)
query_metadata_pl.schema

Schema([('query_name', String),
        ('query_family', Categorical(ordering='physical')),
        ('query_superfamily', Categorical(ordering='physical')),
        ('query_fold', Categorical(ordering='physical')),
        ('query_class', Categorical(ordering='physical')),
        ('n_family', Int64),
        ('n_superfamily', Int64),
        ('n_fold', Int64),
        ('n_class', Int64),
        ('query_scop_id', String)])

In [81]:
query_metadata_pl = query_metadata_pl.with_columns(
    pl.col(col).cast(pl.Categorical("lexical")).alias(col)
    for col in query_scop_cols_minus_name
)
query_metadata_pl.schema

Schema([('query_name', String),
        ('query_family', Categorical(ordering='lexical')),
        ('query_superfamily', Categorical(ordering='lexical')),
        ('query_fold', Categorical(ordering='lexical')),
        ('query_class', Categorical(ordering='lexical')),
        ('n_family', Int64),
        ('n_superfamily', Int64),
        ('n_fold', Int64),
        ('n_class', Int64),
        ('query_scop_id', String)])

In [102]:
%%file polars_utils.py

import polars as pl
import s3fs


def _to_s3(writer, filename):
    fs = s3fs.S3FileSystem()
    with fs.open(filename, mode="wb") as f:
        writer(f)


def _to_filename(writer, filename):
    if filename.startswith("s3://"):
        # Writing Parquet files with arrow can be troublesome, need to use s3fs explicitly
        _to_s3(writer, pq)
    else:
        writer(pq)


def write_parquet(df: pl.DataFrame, pq: str, verbose=False):
    if verbose:
        print(f"\nWriting {df.height} rows and {df.width} columns to {pq} ...")
    _to_filename(df.write_parquet, pq)
    if verbose:
        print("\tDone.")


def sink_parquet(df: pl.LazyFrame, pq: str, verbose=False):
    if verbose:
        print(
            f"\nWriting {df.select(pl.len()).collect().item()} rows and {len(df.columns)} columns to {pq} ..."
        )
    _to_filename(df.sink_parquet, pq)
    print("\tDone.")

Overwriting polars_utils.py


In [103]:
# query_metadata_pl.write_parquet?

In [104]:
from polars_utils import write_parquet

In [105]:
write_parquet(
    query_metadata_pl,
    "s3://seanome-kmerseek/scope-benchmark/reference_files/scop.e.2.08.query_metadata.pq",
)

#### -> This didn't work ... still 'physical' ordering

In [87]:
query_metadata_pl2 = pl.read_parquet(
    "s3://seanome-kmerseek/scope-benchmark/reference_files/scop.e.2.08.query_metadata.pq"
)
query_metadata_pl2.schema

Schema([('query_name', String),
        ('query_family', Categorical(ordering='physical')),
        ('query_superfamily', Categorical(ordering='physical')),
        ('query_fold', Categorical(ordering='physical')),
        ('query_class', Categorical(ordering='physical')),
        ('n_family', Int64),
        ('n_superfamily', Int64),
        ('n_fold', Int64),
        ('n_class', Int64),
        ('query_scop_id', String)])

## Make match metadata, which doesn't have `n_{family,superfamily,fold,class}`

In [88]:
match_metadata = query_metadata_with_n_groups.copy().reset_index()
match_metadata = match_metadata[query_scop_cols + ["query_scop_id"]]
match_metadata.columns = [x.replace("query", "match") for x in match_metadata.columns]
match_metadata.head()

,match_name,match_family,match_superfamily,match_fold,match_class,match_scop_id
0,d1x3ka_ a.1.1.0 (A:) automated matches {Tokuna...,a.1.1.0,a.1.1,a.1,a,d1x3ka_
1,d1x46a_ a.1.1.0 (A:) automated matches {Tokuna...,a.1.1.0,a.1.1,a.1,a,d1x46a_
2,d2bk9a_ a.1.1.0 (A:) automated matches {Fruit ...,a.1.1.0,a.1.1,a.1,a,d2bk9a_
3,d2c0ka_ a.1.1.0 (A:) automated matches {Gaster...,a.1.1.0,a.1.1,a.1,a,d2c0ka_
4,d2ig3a_ a.1.1.0 (A:) automated matches {Campyl...,a.1.1.0,a.1.1,a.1,a,d2ig3a_


In [89]:
match_metadata.to_parquet(
    "s3://seanome-kmerseek/scope-benchmark/reference_files/scop.e.2.08.match_metadata.pq",
    index=False,
)

In [90]:
match_metadata_pl = pl.read_parquet(
    "s3://seanome-kmerseek/scope-benchmark/reference_files/scop.e.2.08.match_metadata.pq"
)

## Make generic metadata, without `query_` prefix

In [91]:
scop_metadata_with_n_groups = query_metadata_with_n_groups.copy().reset_index()
scop_metadata_with_n_groups.columns = [
    x.split("query_")[-1] for x in scop_metadata_with_n_groups.columns
]
scop_metadata_with_n_groups.head()

,scop_id,name,family,superfamily,fold,class,n_family,n_superfamily,n_fold,n_class
0,d1x3ka_,d1x3ka_ a.1.1.0 (A:) automated matches {Tokuna...,a.1.1.0,a.1.1,a.1,a,22,57,62,2644
1,d1x46a_,d1x46a_ a.1.1.0 (A:) automated matches {Tokuna...,a.1.1.0,a.1.1,a.1,a,22,57,62,2644
2,d2bk9a_,d2bk9a_ a.1.1.0 (A:) automated matches {Fruit ...,a.1.1.0,a.1.1,a.1,a,22,57,62,2644
3,d2c0ka_,d2c0ka_ a.1.1.0 (A:) automated matches {Gaster...,a.1.1.0,a.1.1,a.1,a,22,57,62,2644
4,d2ig3a_,d2ig3a_ a.1.1.0 (A:) automated matches {Campyl...,a.1.1.0,a.1.1,a.1,a,22,57,62,2644


In [92]:
scop_metadata_with_n_groups.dtypes

scop_id            object
name               object
family           category
superfamily      category
fold             category
class            category
n_family            int64
n_superfamily       int64
n_fold              int64
n_class             int64
dtype: object

In [93]:
scop_metadata_with_n_groups.to_parquet(
    "s3://seanome-kmerseek/scope-benchmark/reference_files/scop.e.2.08.metadata.pq",
    index=False,
)